## Let's decompose Berkshire Hathaway, and see what is driving the returns
Corresponds to lab 201

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import edhec_risk_kit_201 as erk
import statsmodels.api as sm
import numpy as np

In [5]:
brka_d = pd.read_csv('data/brka_d_ret.csv', parse_dates = True, index_col = 0)
brka_d.head()

,BRKA
DATE,
1990-01-02,-0.005764
1990-01-03,0.000000
1990-01-04,0.005797
1990-01-05,-0.005764
1990-01-08,0.000000


In [7]:
#Take our daily data, and change it to monthly by compounding daily into monthly
brka_m = brka_d.resample('M').apply(erk.compound).to_period('M')
brka_m.head()

,BRKA
DATE,
1990-01,-0.140634
1990-02,-0.030852
1990-03,-0.069204
1990-04,-0.003717
1990-05,0.067164


In [8]:
fff = erk.get_fff_returns()
fff.head()
#gives excess returns, and the two factor mimicking portfolios, and the risk free rate
#SMB is small - big (long small short big), HML is high minus low (long value short growth)

,Mkt-RF,SMB,HML,RF
1926-07,0.0296,-0.0230,-0.0287,0.0022
1926-08,0.0264,-0.0140,0.0419,0.0025
1926-09,0.0036,-0.0132,0.0001,0.0023
1926-10,-0.0324,0.0004,0.0051,0.0032
1926-11,0.0253,-0.0020,-0.0035,0.0031


Next, we need to decompose the observed BRKA 1990-May 2012 as in Ang(2014) into the portion that's due to the market and the rest that is not due to the market, using the CAPM as the explanatory model.

i.e.

$$ R_{brka,t} - R_{f,t} = \alpha + \beta(R_{mkt,t} - R_{f,t}) + \epsilon_t $$

We can use the `stats.api` for the linear regression as follows:

In [12]:
brka_excess = brka_m["1990":"2012-05"] - fff.loc["1990":"2012-05", ['RF']].values
mkt_excess = fff.loc["1990":"2012-05",['Mkt-RF']]
exp_var = mkt_excess.copy()
exp_var["Constant"] = 1
lm = sm.OLS(brka_excess, exp_var).fit()

In [13]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   BRKA   R-squared:                       0.154
Model:                            OLS   Adj. R-squared:                  0.150
Method:                 Least Squares   F-statistic:                     48.45
Date:                Tue, 26 Jan 2021   Prob (F-statistic):           2.62e-11
Time:                        09:08:40   Log-Likelihood:                 388.47
No. Observations:                 269   AIC:                            -772.9
Df Residuals:                     267   BIC:                            -765.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.5402      0.078      6.961      0.000       0.387       0.693
Constant       0.0061      0.004      1.744      0.082      -0.001       0.013
==============================================================================
Omnibus:                       45.698   Durbin-Watson:                   2.079
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              102.573
Skew:                           0.825   Prob(JB):                     5.33e-23
Kurtosis:                       5.535   Cond. No.                         22.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## CAPM benchmark interpretation
This summary says that the CAPM benchmark consists of 46 cents in T-bills (risk free) and 54 dcents in the market. 
    AKA: each dollar in Berkshire hathway is equivalent to 46 cents in T-Bills and 54 cents in the market.
    RELATIVE to this, the Berkshire fund is adding .61% (per month) of alpha. (although degree of significance is not super high)

In [15]:
exp_var["Value"] = fff.loc["1990":"2012-05",['HML']]
exp_var["Size"] = fff.loc["1990":"2012-05",['SMB']]
exp_var.head()

,Mkt-RF,Constant,Value,Size
1990-01,-0.0785,1,0.0087,-0.0129
1990-02,0.0111,1,0.0061,0.0103
1990-03,0.0183,1,-0.0290,0.0152
1990-04,-0.0336,1,-0.0255,-0.0050
1990-05,0.0842,1,-0.0374,-0.0257


In [16]:
lm = sm.OLS(brka_excess, exp_var).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   BRKA   R-squared:                       0.290
Model:                            OLS   Adj. R-squared:                  0.282
Method:                 Least Squares   F-statistic:                     36.06
Date:                Tue, 26 Jan 2021   Prob (F-statistic):           1.41e-19
Time:                        09:13:45   Log-Likelihood:                 412.09
No. Observations:                 269   AIC:                            -816.2
Df Residuals:                     265   BIC:                            -801.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.6761      0.074      9.155      0.000       0.531       0.821
Constant       0.0055      0.003      1.679      0.094      -0.001       0.012
Value          0.3814      0.109      3.508      0.001       0.167       0.595
Size          -0.5023      0.101     -4.962      0.000      -0.702      -0.303
==============================================================================
Omnibus:                       42.261   Durbin-Watson:                   2.146
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               67.954
Skew:                           0.904   Prob(JB):                     1.75e-15
Kurtosis:                       4.671   Cond. No.                         37.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

These coefficients imply that Berkshire has a significant Value tilt, and a NEGATIVE tilt on size (indicating that they tend to invest in large companies)

So the new answer to the question "If I give $100 to Berkshire Hathway, where is the money going?"

    $67 into the market, $33 into bills
    $38 in Value stocks and SHORT $38 in Growth stocks
    Short $50 in small cap stocks and $50 in large cap stocks
But if you did this same exact thing, you'd end up underperforming by about 55 basis points per month

# Exercises

In [17]:
results = erk.regress(brka_excess, mkt_excess)
results.params

Mkt-RF    0.540175
Alpha     0.006133
dtype: float64

In [29]:
brka_excess = brka_m["2012":] - fff.loc['2012':, ['RF']].values
mkt_excess = fff.loc['2012':][['Mkt-RF', 'HML', 'SMB']]

In [35]:
new_results = erk.regress(brka_excess, mkt_excess)

In [38]:
new_results.params

Mkt-RF    0.867331
HML       0.448694
SMB      -0.365786
Alpha     0.003217
dtype: float64